# StockPulse AI - Demo Notebook

This notebook demonstrates the multi-agent stock intelligence system step by step.

## Prerequisites
- Python 3.9+
- Groq API Key
- IndianAPI Key (optional)

## 1. Setup Environment

In [ ]:
# Install dependencies (uncomment if needed)
# !pip install -r requirements.txt

In [ ]:
import os
import sys
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Verify API key is set
if not os.getenv("GROQ_API_KEY"):
    print("⚠️ GROQ_API_KEY not found. Please set it in .env file")
else:
    print("✅ GROQ_API_KEY loaded")

if os.getenv("INDIAN_API_KEY"):
    print("✅ INDIAN_API_KEY loaded (live stock prices enabled)")
else:
    print("ℹ️ INDIAN_API_KEY not set (using web fallback for prices)")

## 2. Initialize the Agents

In [ ]:
from utils import get_research_llm, get_analyst_llm, get_settings
from agents import DataCollectorAgent, AnalystAgent

# Get settings
settings = get_settings()
print(f"Research Model: {settings.research_model}")
print(f"Analyst Model: {settings.analyst_model}")

# Initialize LLMs
research_llm = get_research_llm(temperature=settings.temperature)
analyst_llm = get_analyst_llm(temperature=settings.temperature)

# Initialize Agents
data_collector = DataCollectorAgent(research_llm)
analyst = AnalystAgent(analyst_llm)

print("\n✅ Agents initialized successfully!")

## 3. Data Collection (Agent 1)

The Data Collector agent gathers information from multiple sources:
- Wikipedia
- News search
- Stock APIs (IndianAPI or web fallback)

In [ ]:
# Choose a company to analyze
COMPANY_NAME = "Infosys"  # Try: "Tesla", "Reliance", "TCS"

print(f"🔍 Collecting data for: {COMPANY_NAME}")
print("=" * 50)

# Run data collection
raw_data = data_collector.collect_data(COMPANY_NAME)

print(f"\n✅ Data collected!")
print(f"Data Quality: {raw_data.get('data_quality', 'unknown')}")
print(f"Sources found: {len(raw_data.get('sources', []))}")

In [ ]:
# View collected data
print("📊 COLLECTED DATA:")
print("=" * 50)

# Company Info
company_info = raw_data.get("company_info", {})
print(f"\n📋 Company Description:")
print(company_info.get("description", "N/A")[:300] + "...")

# Stock Data
stock = raw_data.get("stock_performance", {})
print(f"\n💰 Stock Performance:")
print(f"  Price: {stock.get('current_price', 'N/A')}")
print(f"  Change: {stock.get('price_change_pct', 'N/A')}")
print(f"  Trend: {stock.get('trend', 'N/A')}")
print(f"  Source: {stock.get('source', 'N/A')}")

# News
news = raw_data.get("news", [])
print(f"\n📰 Recent News ({len(news)} articles):")
for article in news[:3]:
    print(f"  • {article.get('title', 'N/A')[:60]}...")

## 4. Analysis (Agent 2)

The Analyst agent uses LLM to generate insights from the collected data.

In [ ]:
print(f"🧠 Analyzing data for: {COMPANY_NAME}")
print("=" * 50)

# Run analysis
analysis_results = analyst.analyze(raw_data)

print("\n✅ Analysis complete!")

In [ ]:
# View analysis results
print("📊 ANALYSIS RESULTS:")
print("=" * 50)

print(f"\n📋 Executive Summary:")
print(analysis_results.get("executive_summary", "N/A"))

print(f"\n💡 Market Insights:")
print(analysis_results.get("market_insights", "N/A"))

print(f"\n⚖️ Risks & Opportunities:")
print(analysis_results.get("risks_opportunities", "N/A"))

## 5. Full Workflow with Orchestrator

The Orchestrator manages the complete workflow using LangGraph, maintaining shared state between agents.

In [ ]:
from graph import Orchestrator

# Initialize orchestrator
orchestrator = Orchestrator(data_collector, analyst)

print("✅ Orchestrator initialized with LangGraph workflow")

In [ ]:
# Run complete workflow
COMPANY_NAME = "Tesla"  # Try a different company

print(f"🚀 Running full workflow for: {COMPANY_NAME}")
print("=" * 60)

results = orchestrator.run(COMPANY_NAME)

print("\n" + "=" * 60)
print("✅ WORKFLOW COMPLETE")

In [ ]:
# Display final results
print(f"\n📊 FINAL RESULTS FOR: {results.get('company_name')}")
print("=" * 60)

# Stock info
stock = results.get("raw_data", {}).get("stock_performance", {})
print(f"\n💰 Stock: {stock.get('current_price', 'N/A')} ({stock.get('price_change_pct', 'N/A')})")
print(f"📈 Trend: {stock.get('trend', 'N/A')}")
print(f"🎯 Recommendation: {stock.get('recommendation', 'N/A')}")

# Analysis
analysis = results.get("analysis_results", {})
print(f"\n📋 Summary:")
print(analysis.get("executive_summary", "N/A")[:500])

## 6. State & Context Flow

The orchestrator maintains context between agent calls using `AgentState`:

In [ ]:
# Show the state structure
print("📦 AgentState Structure:")
print("=" * 40)
for key, value in results.items():
    if isinstance(value, dict):
        print(f"\n{key}: (dict with {len(value)} keys)")
        for k in list(value.keys())[:5]:
            print(f"  - {k}")
    else:
        print(f"\n{key}: {value}")

---

## Summary

This notebook demonstrated:

1. **Data Collection** - Gathering info from Wikipedia, news, and stock APIs
2. **Analysis** - LLM-powered insight generation
3. **Orchestration** - LangGraph workflow with shared state
4. **Context Flow** - How `AgentState` maintains memory between calls

For the full UI experience, run:
```bash
streamlit run ui/app.py
```